In [1]:
import os
import pandas as pd
import requests

def newParameters():
    # get file
    df = pd.read_excel('./produitStructure.xlsx',sheet_name='Autocall')
    
    def newParameterTable(df,col,name):
        res=pd.DataFrame()
        x=df[col].fillna('NONE')
        res['label']=[y.strip().upper() for y in x.unique()]
        res['typeLabel']=name
        return res
    
    df_res_1=newParameterTable(df,'Type','produit')
    df_res_2=newParameterTable(df,'clientele_cible','clientele')
    df_res_3=newParameterTable(df,'Emetteur','emetteur')
    df_res_4=newParameterTable(df,'typePanier','panier')
    df_res_5=newParameterTable(df,'Capital garanti','protection')
    df_res_6=newParameterTable(df,'Type de barrière','barriere')
    df_res_7=newParameterTable(df,'Remuneration','remuneration')
    df_res_8=newParameterTable(df,'Coupon ou Participation','garantie')
    df_res_9=newParameterTable(df,'Type de produit','produit2')
    df_res_10=newParameterTable(df,'Devise','devise')
    df_res_11=newParameterTable(df,'Periode','periodeRemb')

    df_res_tot=pd.concat([df_res_1,df_res_2,df_res_3,df_res_4,df_res_5,df_res_6,df_res_7,df_res_8,df_res_9,df_res_10,df_res_11])

    json_data = df_res_tot.to_json(orient='records')
    
    # api-endpoint
    URL = "http://127.0.0.1:5001/api/setparameters"

    # sending get request and saving the response as response object
    r = requests.post(url = URL, json = json_data)
    
    # extracting data in json format
    print('Parameters', r)

def newUsers():
    df=pd.read_excel('./users.xlsx') 
    js_input=df.to_json(orient='records')

    # api-endpoint
    URL = "http://127.0.0.1:5001/api/setusers"

    # defining a params dict for the parameters to be sent to the API
    json = js_input
    
    # sending get request and saving the response as response object
    r = requests.post(url = URL, json = json)
    
    # extracting data in json format
    print('Users:', r)

def newInstruments():
    df_inst = pd.read_excel('./produitStructure.xlsx',sheet_name='TICKER')
    columns_renamed = {
        'CODE ISIN': 'isin',
        'TICKER': 'ticker',
        'Sous-jacent': 'shortName',
    }

    df_inst.rename(columns=columns_renamed, inplace=True)

    INSTRUMENT_final = df_inst[columns_renamed.values()]
    INSTRUMENT_final = INSTRUMENT_final.fillna('')

    js_input=INSTRUMENT_final.to_json(orient='records')

    # api-endpoint
    URL = "http://127.0.0.1:5001/api/setinstruments"

    # sending get request and saving the response as response object
    r = requests.post(url = URL, json = js_input)
    
    # extracting data in json format
    print("Instruments", r)

def newStructuredProducts():
    produit_structure = pd.read_excel('./produitStructure.xlsx', sheet_name="Autocall")
    columns_renamed = {
        'Code ISIN': 'isin',
        'Ticker du sous-jacent': 'ticker',
        'Devise': 'devise',
        'Intitulé commercial': 'shortName',
        'typeCommentaire': 'description',
        'Montant Souscrit': 'investissement',
        'Type de barrière': 'barriere',
        'Type': 'produit',
        'Emetteur': 'emetteur',
        'Maturité': 'maturite',
        'Type de produit': 'produit2',
        'clientele_cible': 'clientele',
        'Remuneration': 'remuneration',
        'typePanier': 'panier',

        'Date credit en compte': 'dateAchat',
        'Date d\'émission': 'dateEmission',
        'Date d\'échéance': 'dateEchFinale',
        'Date de remboursement effectif': 'dateRembEff',

        #'': 'periodeRemb',
        'Periode': 'periodeRemb',
        'Période de non Call': 'periodeNocall'
    }

    produit_structure.rename(columns=columns_renamed, inplace=True)

    for col in columns_renamed.values():
        if col not in produit_structure.columns:
            produit_structure[col] = None
        

    PS_final = produit_structure[columns_renamed.values()]
    PS_final = PS_final.fillna('')
    PS_final['dateAchat'] = PS_final['dateAchat'].dt.strftime('%d/%m/%Y').astype(str).replace('nan', '01/01/1900')
    PS_final['dateEchFinale'] = PS_final['dateEchFinale'].dt.strftime('%d/%m/%Y').astype(str).replace('nan', '01/01/1900')
    PS_final['dateEmission'] = PS_final['dateEmission'].dt.strftime('%d/%m/%Y').astype(str).replace('nan', '01/01/1900')
    PS_final['dateRembEff'] = PS_final['dateRembEff'].dt.strftime('%d/%m/%Y').astype(str).replace('nan', '01/01/1900')
    
    

    #PS_final["periodeRemb"] = PS_final["periodeRemb"].replace('','0').astype(int)
    PS_final["maturite"] = PS_final["maturite"].replace('','0').astype(int)
    PS_final["investissement"] = PS_final["investissement"].replace('','0').astype(int)
    PS_final["maturite"] = PS_final["maturite"].replace('','0').astype(int)

    # From the column description, remove the html tags
    PS_final['description'] = PS_final['description'].str.replace('<[^<]+?>', '')
    # From the column description, remvoe characters that are between _
    PS_final['description'] = PS_final['description'].str.replace('_.*_', '')
    
    def normParam(x):
        return x.strip().upper() if len(x)>0 else 'NONE'

    for col in ["emetteur","produit","produit2","clientele","remuneration","panier","devise","barriere","periodeRemb"]:
        PS_final[col]=PS_final[col].apply(normParam)

    json_input = PS_final.to_json(orient='records')

    # api-endpoint
    URL = "http://127.0.0.1:5001/api/setproducts"

    r = requests.post(url = URL, json = json_input)
    print("Structured Products", r)

    # Rajout des managers
    portfolios = pd.read_csv('./BANKVISTA_portfolios_20230503_20230504-190004.csv', sep=';')
    positions = pd.read_csv('./BANKVISTA_positions_20230503_20230504-190003.csv', sep=';')
    
    merge_df=pd.merge(positions[['PortfolioId','ISIN']], portfolios[['AccountID','RelationshipManager']], right_on='AccountID', left_on='PortfolioId', how='inner')
    isin_list=PS_final['isin'].tolist()
    managers=merge_df[merge_df['ISIN'].isin(isin_list)]['RelationshipManager'].unique()
    managers_df = pd.DataFrame(columns=['name', 'surname', 'email', 'role', 'password'])
    for manager in managers:
        role = 'FRONT'
        if manager == 'MARTIN LIONEL':
            role = 'ADMIN'
        managers_df = pd.concat([managers_df,pd.DataFrame({'name': manager.split(' ')[0], 'surname': manager.split(' ')[1].title(), 'email': manager.split(' ')[0].lower() + '.' + manager.split(' ')[1].lower() + '@massena.lu', 'role': role, 'password': 'massena'}, index=[0])], ignore_index=True)

    js_input=managers_df.to_json(orient='records')

    # api-endpoint
    URL = "http://127.0.0.1:5001/api/setusers"

    # defining a params dict for the parameters to be sent to the API
    json = js_input
    
    # sending get request and saving the response as response object
    r = requests.post(url = URL, json = json)
    
    # extracting data in json format
    print('Managers:', r)

def newSsJacents():
    ticker = pd.read_excel('./produitStructure.xlsx', sheet_name="TICKER")
    columns_renamed = {'CODE ISIN': 'isin','TICKER': 'ticker','Sous-jacent': 'shortName','Prix': 'price','DatePrix': 'priceDate'}

    ticker.rename(columns=columns_renamed, inplace=True)
                    
    INSTRUMENT_final = ticker[columns_renamed.values()]
    INSTRUMENT_final = INSTRUMENT_final.fillna('')

    # Transform priceDate to DateTime
    INSTRUMENT_final['priceDate'] = pd.to_datetime(INSTRUMENT_final['priceDate'], format='%d/%m/%Y', errors='coerce')
    INSTRUMENT_final['priceDate'] = INSTRUMENT_final['priceDate'].dt.strftime('%Y-%m-%d').astype(str).replace('nan', '01/01/1900')
    INSTRUMENT_final["price"] = INSTRUMENT_final["price"].replace('','0').astype(float)

    ssjacents = pd.read_excel('./produitStructure.xlsx', sheet_name="SSJACENT")
    ssjacents.columns = ssjacents.loc[581].reset_index(drop=True)
    ssjacents.columns.name = None
    ssjacents['ISIN'] = ssjacents['ISIN'].str.replace('[^a-zA-Z0-9]', '')
    ssjacents = ssjacents.rename(columns={'ISIN': 'isin','NOM': 'shortName'})
    ssjacents_reworked = pd.merge(INSTRUMENT_final, ssjacents, on='shortName', how='inner')
    ssjacents_reworked[["shortName", "ticker", "isin_x", "isin_y"]]

    ssjacents_reworked = ssjacents_reworked.rename(columns={'isin_x':'idInstrument','isin_y':'idProduit'})
    ssjacents_reworked=ssjacents_reworked[['idProduit','ticker','price','priceDate','strike']]

    ssjacents_reworked=ssjacents_reworked.rename(columns={'idProduit':'isinProduit','ticker':'TickerSj'})

    json_input = ssjacents_reworked.to_json(orient='records')
    # api-endpoint
    URL = "http://127.0.0.1:5001/api/setssjacents"

    r = requests.post(url = URL, json = json_input)
    print(r)

def newEcheances():
    dfEch=pd.read_excel('./produitStructure.xlsx', sheet_name="BARRIERE")

    dfEch=dfEch.rename(columns={'ISIN':'isinProduit','Pourcentage':'Percent','Periode':'Period'})
    dfEch['noncall']=dfEch['noncall'].apply(lambda x: True if x=='OUI'else False)
    dfEch['type']='Remb'

    json_input = dfEch.to_json(orient='records')

    json_input = dfEch.to_json(orient='records')
    # api-endpoint
    URL = "http://127.0.0.1:5001/api/setecheances"

    r = requests.post(url = URL, json = json_input)
    print(r)

def initLoad():
    newParameters()
    newUsers()
    newInstruments()
    newStructuredProducts()
    newSsJacents()
    newEcheances()

if __name__ == "__main__":
    initLoad()

Parameters <Response [500]>
Users: <Response [500]>
Instruments <Response [500]>


/var/folders/5m/jp31_5x11h77nzkg17xz_wfc0000gn/T/ipykernel_83703/3159356561.py:129: FutureWarning: The default value of regex will change from True to False in a future version.
  PS_final['description'] = PS_final['description'].str.replace('<[^<]+?>', '')
/var/folders/5m/jp31_5x11h77nzkg17xz_wfc0000gn/T/ipykernel_83703/3159356561.py:131: FutureWarning: The default value of regex will change from True to False in a future version.
  PS_final['description'] = PS_final['description'].str.replace('_.*_', '')


Structured Products <Response [500]>
Managers: <Response [500]>
<Response [500]>
<Response [500]>


/var/folders/5m/jp31_5x11h77nzkg17xz_wfc0000gn/T/ipykernel_83703/3159356561.py:192: FutureWarning: The default value of regex will change from True to False in a future version.
  ssjacents['ISIN'] = ssjacents['ISIN'].str.replace('[^a-zA-Z0-9]', '')


In [5]:
!python3 --version


Python 3.11.0
